In [1]:
from binance.client import Client
import pandas as pd
import pandas_ta as ta


In [3]:
# Initialize the Binance client without authentication
client = Client()


In [5]:
from datetime import datetime, timedelta

# Fetch the last 40 records of the 5-minute candle for BTCUSDT
symbol = 'BTCUSDT'
interval = '1m'  # Valid interval for 5-minute candles
limit = 60

# Calculate the timestamp for one year ago
one_year_ago = datetime.now() - timedelta(days=365)
one_year_ago_timestamp = int(one_year_ago.timestamp() * 1000)

# Fetch klines data from one year ago until now
klines = []
while True:
    new_klines = client.get_klines(symbol=symbol, interval=interval, startTime=one_year_ago_timestamp, limit=1000)
    if not new_klines:
        break
    klines.extend(new_klines)
    one_year_ago_timestamp = new_klines[-1][0] + 1  # Move to the next timestamp

# Ensure we have data up to the current time
df = klines
klines = klines[:limit]

In [6]:
bkp_df = df

In [7]:
# Convert the data to a pandas DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
df = pd.DataFrame(bkp_df, columns=columns)

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Convert 'high', 'low', 'close', 'volume' columns to decimals
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

df.head(100)

,Open,High,Low,Close,Volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2024-03-12 19:03:00,71228.03,71420.00,71216.00,71393.14,66.22167,1710270239999,4724773.13574920,1942,35.31209000,2519139.71043500,0
2024-03-12 19:04:00,71393.14,71421.10,71352.00,71360.00,44.71778,1710270299999,3192154.84368530,1356,25.41999000,1814580.86539460,0
2024-03-12 19:05:00,71360.01,71497.77,71321.10,71432.08,143.61385,1710270359999,10254125.80757010,2453,99.53279000,7105475.79466370,0
2024-03-12 19:06:00,71432.08,71488.00,71413.06,71417.44,37.30804,1710270419999,2665779.26538340,1309,14.54271000,1039138.03727130,0
2024-03-12 19:07:00,71417.43,71450.20,71383.98,71445.34,52.14702,1710270479999,3724600.70018490,1338,29.30752000,2093216.37150580,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12 20:38:00,70929.50,70980.85,70886.61,70932.01,34.16637,1710275939999,2423662.13210490,1603,21.28937000,1510161.64754610,0
2024-03-12 20:39:00,70932.01,70987.99,70890.00,70987.99,37.45111,1710275999999,2656362.43075790,1260,19.66072000,1394597.34907700,0
2024-03-12 20:40:00,70987.98,71037.50,70956.67,71032.22,65.99197,1710276059999,4685577.52339890,2050,45.47313000,3228523.56547560,0


In [8]:
# Calculate VWAP
df['VWAP'] = ta.vwap(df['High'], df['Low'], df['Close'], df['Volume']).rolling(window=60).mean()
# Calculate 200 EMA
df['200_EMA'] = df['Close'].ewm(span=200, adjust=False).mean()

df.head(100)

,Open,High,Low,Close,Volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,VWAP,200_EMA
timestamp,,,,,,,,,,,,,
2024-03-12 19:03:00,71228.03,71420.00,71216.00,71393.14,66.22167,1710270239999,4724773.13574920,1942,35.31209000,2519139.71043500,0,NaN,71393.140000
2024-03-12 19:04:00,71393.14,71421.10,71352.00,71360.00,44.71778,1710270299999,3192154.84368530,1356,25.41999000,1814580.86539460,0,NaN,71392.810249
2024-03-12 19:05:00,71360.01,71497.77,71321.10,71432.08,143.61385,1710270359999,10254125.80757010,2453,99.53279000,7105475.79466370,0,NaN,71393.200993
2024-03-12 19:06:00,71432.08,71488.00,71413.06,71417.44,37.30804,1710270419999,2665779.26538340,1309,14.54271000,1039138.03727130,0,NaN,71393.442177
2024-03-12 19:07:00,71417.43,71450.20,71383.98,71445.34,52.14702,1710270479999,3724600.70018490,1338,29.30752000,2093216.37150580,0,NaN,71393.958573
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12 20:38:00,70929.50,70980.85,70886.61,70932.01,34.16637,1710275939999,2423662.13210490,1603,21.28937000,1510161.64754610,0,71429.994261,71295.957981
2024-03-12 20:39:00,70932.01,70987.99,70890.00,70987.99,37.45111,1710275999999,2656362.43075790,1260,19.66072000,1394597.34907700,0,71427.234878,71292.893623
2024-03-12 20:40:00,70987.98,71037.50,70956.67,71032.22,65.99197,1710276059999,4685577.52339890,2050,45.47313000,3228523.56547560,0,71424.408557,71290.299856


In [15]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

last_100_df = df.iloc[-1000000:]

# SL_FACTOR = 0.01
# TP_FACTOR = 0.03
SL_FACTOR = 0.5
TP_FACTOR = 0.5

class VWAPStrategy(Strategy):
    def init(self):
        self.vwap = self.data.VWAP

    def next(self):
        # BUY_SIGNAL = self.data.Close[-1] < self.vwap[-1] and self.data.Close[-1] > self.data['200_EMA'][-1]
        # SELL_SIGNAL = self.data.Close[-1] > self.vwap[-1] and self.data.Close[-1] < self.data['200_EMA'][-1]
        BUY_SIGNAL = self.data.Close[-1] < self.vwap[-1] 
        SELL_SIGNAL = self.data.Close[-1] > self.vwap[-1]


        if(len(self.orders) > 0):
            for order in self.orders:
                if order.is_contingent == 0:
                    order.cancel()

        if BUY_SIGNAL and self.position.size < 0:
            self.position.close()
            # print("a Close sell position")

        if SELL_SIGNAL and self.position.size > 0:
            self.position.close()

        if self.position.size == 0:
            if BUY_SIGNAL:
                # print('Buy signal')
                self.buy(size=1, limit=self.data.Low[-1], sl=self.data.Low[-1]-(self.data.Low[-1]*SL_FACTOR), tp=self.data.Low[-1]+(self.data.Low[-1]*TP_FACTOR))
                # self.buy(size=1)
            elif SELL_SIGNAL:
                self.sell(size=1, limit=self.data.High[-1], sl=self.data.High[-1]+(self.data.High[-1]*SL_FACTOR), tp=self.data.High[-1]-(self.data.High[-1]*TP_FACTOR))
                # self.sell(size=1)


# Run backtest
bt = Backtest(last_100_df, VWAPStrategy, cash=1000000, commission=0.002)
stats = bt.run()
bt.plot()
stats

/Users/yurisa2/petrosa/petrosa-crypto-cloud-vwap/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:139: UserWarning: Data contains too many candlesticks to plot; downsampling to '1h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


Start                     2024-03-12 19:03:00
End                       2025-03-12 19:05:00
Duration                    365 days 00:02:00
Exposure Time [%]                    99.98478
Equity Final [$]                  998360.0358
Equity Peak [$]                  1011686.5191
Commissions [$]                    31350.1342
Return [%]                             -0.164
Buy & Hold Return [%]                15.67816
Return (Ann.) [%]                     -0.1593
Volatility (Ann.) [%]                 3.91205
CAGR [%]                               -0.164
Sharpe Ratio                         -0.04072
Sortino Ratio                        -0.05859
Calmar Ratio                          -0.0597
Max. Drawdown [%]                    -2.66812
Avg. Drawdown [%]                    -0.10593
Max. Drawdown Duration      236 days 19:59:00
Avg. Drawdown Duration        3 days 09:52:00
# Trades                                   95
Win Rate [%]                            100.0
Best Trade [%]                    